In [12]:
import requests
import pandas as pd
import redshift_connector
from dotenv import dotenv_values
import psycopg2
from io import StringIO
import sqlalchemy as sa
from sqlalchemy.engine.url import URL
from os import environ as env


data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com


KeyError: 'HOST'

In [3]:
function = 'TIME_SERIES_WEEKLY'
# Big Five Tech: Google, Amazon, Meta, Apple, and Microsoft (GAMAM)
symbols = ['GOOG', 'AMZN', 'METV', 'AAPL', 'MSFT']
api_key = 'WNHSBPLUX5B8HNMZ'

df = pd.DataFrame()

for symbol in symbols:
    url = f'https://www.alphavantage.co/query?function={function}&symbol={symbol}&apikey={api_key}'
    r = requests.get(url)
    data = r.json()

    symbol_df = pd.DataFrame(data['Weekly Time Series'])
    symbol_df = symbol_df.T
    symbol_df.reset_index(inplace=True)
    symbol_df.rename(columns={'index':'week_from', '1. open':'open', '2. high':'high', '3. low': 'low', 
                       '4. close':'close', '5. volume':'volume'}, inplace=True)
    
    symbol_df['symbol'] = data['Meta Data']['2. Symbol']

    symbol_df['open'] = pd.to_numeric(symbol_df['open'])
    symbol_df['close'] = pd.to_numeric(symbol_df['close'])
    symbol_df['avg'] = (symbol_df['open'] + symbol_df['close'])/2
    symbol_df['pk'] = symbol_df['symbol']+symbol_df['week_from']

    # symbol_df['avg'] = symbol_df[['close', 'open']].mean(axis=1)
    
    df = pd.concat([df, symbol_df])

In [4]:
url = URL.create(
drivername='redshift+redshift_connector', # indicate redshift_connector driver and dialect will be used
host=env_vars['HOST'], # Amazon Redshift host
port=int(env_vars['PORT']), # Amazon Redshift port
database=env_vars['DATABASE'], # Amazon Redshift database
username=env_vars['USER'], # Amazon Redshift username
password=env_vars['PASSWORD'] # Amazon Redshift password
)

engine = sa.create_engine(url)


In [5]:
# Connect to Redshift using psycopg2
conn = psycopg2.connect(
    host=env_vars['HOST'],
    port=int(env_vars['PORT']),
    database=env_vars['DATABASE'],
    user=env_vars['USER'],
    password=env_vars['PASSWORD']
 )

cursor = conn.cursor()

df.to_sql(name='stage',
          con=engine,
          if_exists='append',
          index=False)
          # method=callable)


In [20]:
# Connect to Redshift using psycopg2
conn = psycopg2.connect(
    host=env_vars['HOST'],
    port=int(env_vars['PORT']),
    database=env_vars['DATABASE'],
    user=env_vars['USER'],
    password=env_vars['PASSWORD']
 )

cursor = conn.cursor()

sql_transaction = ''' begin transaction;
                      
                      CREATE TABLE IF NOT EXISTS laureanoengulian_coderhouse.big_five_weekly(
                      "week_from" varchar(256) not null,
                      "open" float8 not null,
                      "high" varchar(256) not null,
                      "low" varchar(256) not null,
                      "close" float8 not null,
                      "volume" varchar(256) not null,
                      "symbol" varchar(256) not null,
                      "avg" float8 not null,
                      "pk" varchar(256));


                      delete from laureanoengulian_coderhouse.big_five_weekly using laureanoengulian_coderhouse.stage 
                      where big_five_weekly.pk = stage.pk;

                      insert into laureanoengulian_coderhouse.big_five_weekly
                      select * from laureanoengulian_coderhouse.stage;

                      end transaction;
                  '''

cursor.execute(sql_transaction)
conn.commit()

drop_tmp = '''drop table if exists laureanoengulian_coderhouse.stage;'''

cursor.execute(drop_tmp)
conn.commit()

In [2]:
# Connect to Redshift using psycopg2
conn = psycopg2.connect(
    host=env['HOST'],
    port=int(env['PORT']),
    database=env['DATABASE'],
    user=env['USER'],
    password=env['PASSWORD']
 )

cursor = conn.cursor()
cursor.execute(f"""
SELECT
  count(pk), count(distinct pk)
FROM
  laureanoengulian_coderhouse.big_five_weekly;
""")
# resultado = cursor.fetchall()
print(", ".join(map(lambda x: str(x), cursor.fetchall())))
cursor.close()

KeyError: 'HOST'

In [16]:
df.head()

,week_from,open,high,low,close,volume,symbol,avg,pk
0,2023-06-28,121.4656,122.7200,116.9100,121.08,70159820,GOOG,121.2728,GOOG2023-06-28
1,2023-06-23,123.5350,125.1750,119.6000,123.02,95665041,GOOG,123.2775,GOOG2023-06-23
2,2023-06-16,123.3950,126.7000,122.1600,124.06,147453173,GOOG,123.7275,GOOG2023-06-16
3,2023-06-09,124.6100,129.5500,122.0100,122.87,121430664,GOOG,123.7400,GOOG2023-06-09
4,2023-06-02,126.2900,126.7450,122.8900,125.23,113094630,GOOG,125.7600,GOOG2023-06-02


In [17]:
# # Connect to Redshift using psycopg2
# conn = psycopg2.connect(
#     host=env_vars['HOST'],
#     port=int(env_vars['PORT']),
#     database=env_vars['DATABASE'],
#     user=env_vars['USER'],
#     password=env_vars['PASSWORD']
#  )

# cursor = conn.cursor()

# # Eliminate the table if exists
# cleaner_query = '''DROP TABLE IF EXISTS laureanoengulian_coderhouse.big_five_weekly;'''
# cursor.execute(cleaner_query)
# conn.commit()

# # Create table if not exists
# create_table_query = '''CREATE TABLE IF NOT EXISTS laureanoengulian_coderhouse.big_five_weekly(
#                         "week_from" date not null,
#                         "open" decimal(38, 4) not null,
#                         "high" decimal(38, 4) not null,
#                         "low" decimal(38, 4) not null,
#                         "close" decimal(38, 4) not null,
#                         "volume" bigint not null,
#                         "symbol" varchar(15) not null,
#                         "avg" decimal(38,4) not null,
#                         "pk" varchar(60));
#                      '''

# # Create table if not exists
# create_table_query = '''CREATE TABLE IF NOT EXISTS laureanoengulian_coderhouse.big_five_weekly(
#                         "week_from" varchar(256) not null,
#                         "open" float8 not null,
#                         "high" varchar(256) not null,
#                         "low" varchar(256) not null,
#                         "close" float8 not null,
#                         "volume" varchar(256) not null,
#                         "symbol" varchar(256) not null,
#                         "avg" float8 not null,
#                         "pk" varchar(256));
#                      '''

# cursor.execute(create_table_query)
# conn.commit()